<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Assessment_SetFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SetFit (Sentence Transformer Fine-tuning): Efficient Few-Shot Learning Without Prompts

## Options
1. Training on GPU, inference on GPU
1. Training on GPU, inference on CPU (recommended)
1. **Training on CPU, inference on CPU (training a bit slow, but this is the default set here)**

## How it works
1. fine-tuning a pretrained Sentence Transformers (ST) on a small number of text pairs, in a contrastive Siamese manner
1. training a classifier head on the embeddings generated from the fine-tuned ST

![](https://raw.githubusercontent.com/huggingface/setfit/main/assets/setfit.png)

## Detailed steps of of contrastive Siamese training for fine-tuning

1. **Embedding Generation**: The input data is passed through a pretrained transformer model, like Sentence-BERT or RoBERTa, to generate embeddings. These embeddings are vector representations of the text data, capturing the semantic nuances in a high-dimensional space.

1. **Contrastive Loss Calculation**: In contrastive training, the goal is to adjust the embeddings so that similar texts (texts with the same label) are closer together in the embedding space, and dissimilar texts (texts with different labels) are farther apart. Involves pairs of texts, where the model tries to minimize the distance between pairs of similar texts while ensuring that pairs of dissimilar texts are separated by at least a margin.

1. **Model Training**: The model is trained by optimizing this contrastive loss across all selected texts in the dataset. During training, the parameters of the model (or a portion of the model if using fine-tuning) are adjusted to reduce the loss, thereby learning to generate embeddings that effectively group similar texts together and push dissimilar texts apart.

1. **Outcome**: After this step, the model produces high-quality embeddings that are more useful for the specific classification or analysis tasks because they better represent the differences and similarities as per the task-specific data.

This contrastive training step effectively leverages a small amount of labeled data to teach the model a nuanced understanding of the task at hand, setting a strong foundation for the subsequent fine-tuning steps. This makes SetFit particularly powerful in scenarios where labeled data is scarce but quality embeddings are crucial for performance.

## Links
* Intrduction: https://huggingface.co/blog/setfit
* This code mostly taken from: https://huggingface.co/docs/setfit/quickstart
* Sources with technical details notebooks: https://github.com/huggingface/setfit
  * https://github.com/huggingface/setfit/tree/main/notebooks
* Paper [2209.11055] Efficient Few-Shot Learning Without Prompts: https://arxiv.org/abs/2209.11055




In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install -q setfit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
gc

In [3]:
import setfit
setfit.__version__

'1.0.3'

# Data

In [4]:
positive = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
negative = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [6]:
from datasets import Dataset
ds = Dataset.from_dict({"text": positive + negative, "label": len(positive) * [1] + len(negative) * [0]})
# ds.to_list()

In [7]:
from setfit import sample_dataset

train_dataset = sample_dataset(ds, label_column="label", num_samples=3)
train_dataset.to_list()

[{'text': 'Socio-medical indication for the aid is confirmed.', 'label': 1},
 {'text': 'A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.',
  'label': 0},
 {'text': 'Everyday relevant usage benefits have been determined.',
  'label': 1},
 {'text': 'The socio-medical prerequisites for the prescribed aid supply have been met.',
  'label': 1},
 {'text': 'From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.',
  'label': 0},
 {'text': 'According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.',
  'label': 0}]

# Few Shot Training

In [8]:
%%time

from setfit import SetFitModel
model_id = "BAAI/bge-small-en-v1.5"
model = SetFitModel.from_pretrained(model_id)
model.labels = ["negative", "positive"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


CPU times: user 981 ms, sys: 461 ms, total: 1.44 s
Wall time: 6.41 s


In [9]:
from setfit import Trainer, TrainingArguments

args = TrainingArguments(
    batch_size=16, # even though we have less samples, this makes sense - we train on unique pairs
    num_epochs=1, # Number of epochs to use for contrastive learning
    num_iterations=20, # Number of text pairs to generate for contrastive learning
)

# https://github.com/huggingface/setfit/issues/512#issuecomment-2118679266
args.eval_strategy = args.evaluation_strategy

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset
)

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [10]:
%%time

# this does train on CPU, takes 1,5 minutes for one epoch
trainer.train()

***** Running training *****
  Num unique pairs = 240
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 15


Step,Training Loss


CPU times: user 1min 5s, sys: 2.21 s, total: 1min 7s
Wall time: 1min 8s


In [11]:
trainer.evaluate(train_dataset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running evaluation *****


{'accuracy': 1.0}

In [12]:
trainer.evaluate(ds)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
***** Running evaluation *****


{'accuracy': 0.8}

# Trying it out

In [13]:
model.predict(negative)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['negative', 'negative', 'negative', 'negative', 'negative']

In [14]:
positive

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.',
 'The socio-medical prerequisites for the prescribed aid supply have been met.',
 'Everyday relevant usage benefits have been determined.',
 'Socio-medical indication for the aid is confirmed.',
 'Contraindications have been excluded; there are no contraindications for the use of the requested aid.']

In [15]:
model.predict(positive)

['negative', 'positive', 'positive', 'positive', 'negative']

In [16]:
%%time

# give it a shot, what is your example
model.predict([
    "Give them what they want.",
    "They get nothing",
    "Are you kidding me?"
])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 48.1 ms, sys: 2.01 ms, total: 50.1 ms
Wall time: 50.9 ms


['positive', 'negative', 'negative']

In [17]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Saving model

In [18]:
model_name = "setfit-bge-small-v1.5-sst2-8-shot"

In [19]:
model.save_pretrained(model_name)

In [20]:
!ls -lh {model_name}

total 129M
drwxr-xr-x 2 root root 4.0K Aug  9 07:48 1_Pooling
drwxr-xr-x 2 root root 4.0K Aug  9 07:48 2_Normalize
-rw-r--r-- 1 root root  706 Aug  9 07:48 config.json
-rw-r--r-- 1 root root  201 Aug  9 07:48 config_sentence_transformers.json
-rw-r--r-- 1 root root   85 Aug  9 07:48 config_setfit.json
-rw-r--r-- 1 root root 3.9K Aug  9 07:48 model_head.pkl
-rw-r--r-- 1 root root 128M Aug  9 07:48 model.safetensors
-rw-r--r-- 1 root root  349 Aug  9 07:48 modules.json
-rw-r--r-- 1 root root 8.5K Aug  9 07:48 README.md
-rw-r--r-- 1 root root   52 Aug  9 07:48 sentence_bert_config.json
-rw-r--r-- 1 root root  695 Aug  9 07:48 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Aug  9 07:48 tokenizer_config.json
-rw-r--r-- 1 root root 695K Aug  9 07:48 tokenizer.json
-rw-r--r-- 1 root root 227K Aug  9 07:48 vocab.txt


In [21]:
# download to local machine
!tar czvf {model_name}.tgz {model_name}

setfit-bge-small-v1.5-sst2-8-shot/
setfit-bge-small-v1.5-sst2-8-shot/1_Pooling/
setfit-bge-small-v1.5-sst2-8-shot/1_Pooling/config.json
setfit-bge-small-v1.5-sst2-8-shot/config_setfit.json
setfit-bge-small-v1.5-sst2-8-shot/vocab.txt
setfit-bge-small-v1.5-sst2-8-shot/modules.json
setfit-bge-small-v1.5-sst2-8-shot/tokenizer.json
setfit-bge-small-v1.5-sst2-8-shot/model_head.pkl
setfit-bge-small-v1.5-sst2-8-shot/sentence_bert_config.json
setfit-bge-small-v1.5-sst2-8-shot/tokenizer_config.json
setfit-bge-small-v1.5-sst2-8-shot/model.safetensors
setfit-bge-small-v1.5-sst2-8-shot/2_Normalize/
setfit-bge-small-v1.5-sst2-8-shot/config_sentence_transformers.json
setfit-bge-small-v1.5-sst2-8-shot/special_tokens_map.json
setfit-bge-small-v1.5-sst2-8-shot/config.json
setfit-bge-small-v1.5-sst2-8-shot/README.md


# In case you were training on GPU: Loading onto CPU and making inferences

**Only if you were training on GPU and want to try inference on CPU**: Switch to CPU (which also restarts the runtime), upload the tgz saved before and execute *only the following* on CPU

In [22]:
model_name = "setfit-bge-small-v1.5-sst2-8-shot"

In [23]:
!tar xzvf {model_name}.tgz

setfit-bge-small-v1.5-sst2-8-shot/
setfit-bge-small-v1.5-sst2-8-shot/1_Pooling/
setfit-bge-small-v1.5-sst2-8-shot/1_Pooling/config.json
setfit-bge-small-v1.5-sst2-8-shot/config_setfit.json
setfit-bge-small-v1.5-sst2-8-shot/vocab.txt
setfit-bge-small-v1.5-sst2-8-shot/modules.json
setfit-bge-small-v1.5-sst2-8-shot/tokenizer.json
setfit-bge-small-v1.5-sst2-8-shot/model_head.pkl
setfit-bge-small-v1.5-sst2-8-shot/sentence_bert_config.json
setfit-bge-small-v1.5-sst2-8-shot/tokenizer_config.json
setfit-bge-small-v1.5-sst2-8-shot/model.safetensors
setfit-bge-small-v1.5-sst2-8-shot/2_Normalize/
setfit-bge-small-v1.5-sst2-8-shot/config_sentence_transformers.json
setfit-bge-small-v1.5-sst2-8-shot/special_tokens_map.json
setfit-bge-small-v1.5-sst2-8-shot/config.json
setfit-bge-small-v1.5-sst2-8-shot/README.md


In [24]:
!ls -lh {model_name}

total 129M
drwxr-xr-x 2 root root 4.0K Aug  9 07:48 1_Pooling
drwxr-xr-x 2 root root 4.0K Aug  9 07:48 2_Normalize
-rw-r--r-- 1 root root  706 Aug  9 07:48 config.json
-rw-r--r-- 1 root root  201 Aug  9 07:48 config_sentence_transformers.json
-rw-r--r-- 1 root root   85 Aug  9 07:48 config_setfit.json
-rw-r--r-- 1 root root 3.9K Aug  9 07:48 model_head.pkl
-rw-r--r-- 1 root root 128M Aug  9 07:48 model.safetensors
-rw-r--r-- 1 root root  349 Aug  9 07:48 modules.json
-rw-r--r-- 1 root root 8.5K Aug  9 07:48 README.md
-rw-r--r-- 1 root root   52 Aug  9 07:48 sentence_bert_config.json
-rw-r--r-- 1 root root  695 Aug  9 07:48 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Aug  9 07:48 tokenizer_config.json
-rw-r--r-- 1 root root 695K Aug  9 07:48 tokenizer.json
-rw-r--r-- 1 root root 227K Aug  9 07:48 vocab.txt


In [25]:
!pip install -q setfit

In [26]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_name)

In [27]:
%%time

# give it a shot, what is your example
model.predict([
    "Give them what they want.",
    "They get nothing",
    "Are you kidding me?"
])

CPU times: user 50.2 ms, sys: 1.14 ms, total: 51.3 ms
Wall time: 50.3 ms


['positive', 'negative', 'negative']